In [11]:
import pandas as pd
import numpy as np

In [12]:
visa = pd.read_csv("visa.csv", index_col = 0)

visa.groupby("Citizenship")["Decision"].value_counts()

Citizenship  Decision
AFGHANISTAN  Y            3
             N            1
ALBANIA      Y           10
             N            2
ALGERIA      N            2
                         ..
YUGOSLAVIA   N            1
ZAMBIA       Y            3
             N            1
ZIMBABWE     Y           16
             N            1
Name: Decision, Length: 277, dtype: int64

In [13]:
visa.head()

,Category,Citizenship,Application Type,Visa Class,Decision
0,Automotive,ECUADOR,MAILEDIN,EWI,Y
1,Advanced Mfg,INDIA,ONLINE,H-1B,Y
2,Advanced Mfg,PHILIPPINES,ONLINE,L-1,Y
3,Advanced Mfg,INDIA,ONLINE,H-1B,Y
4,Advanced Mfg,INDIA,ONLINE,H-1B,Y


In [14]:
visa["Visa Class"].unique()

array(['EWI', 'H-1B', 'L-1', 'E-2', 'TPS', 'B-2', 'B-1', 'F-1', 'H-2B',
       'VWT', 'Parolee', 'TN', 'Not in USA', 'H-4', 'F-2', 'E-3', 'O-1',
       'H-2A', 'H-1B1', 'E-1', 'G-5', 'A-3', 'A1/A2', 'H-3', 'P-1', 'C-1',
       'L-2', 'VWB', 'J-1', 'O-3', 'O-2', 'J-2', 'G-4', 'H-1A', 'R-1',
       'I', 'H1B', 'N', 'T-1', 'TD', 'Parol', 'AOS/H-1B', 'M-1', 'P-3',
       'K-1'], dtype=object)

- H-1B: work visa for professionals working in US
- L-1: work visa for foreiners opening business in US
- F-1: student visa
- Parolee: a permit for non-US national who does not have valid immigrant visa, but need to reenter

In [15]:
(visa["Visa Class"].value_counts() / len(visa)).head(4)

H-1B       0.851572
L-1        0.036200
F-1        0.028478
Parolee    0.019538
Name: Visa Class, dtype: float64

In [16]:
len(visa["Visa Class"].unique())

45

In [23]:
visa.columns = visa.columns.str.lower()
visa.to_csv("./data/visa.csv")
visa.head()

,category,citizenship,application type,visa class,decision
0,Automotive,ECUADOR,MAILEDIN,EWI,Y
1,Advanced Mfg,INDIA,ONLINE,H-1B,Y
2,Advanced Mfg,PHILIPPINES,ONLINE,L-1,Y
3,Advanced Mfg,INDIA,ONLINE,H-1B,Y
4,Advanced Mfg,INDIA,ONLINE,H-1B,Y


In [31]:
pd.pivot_table(visa, index=["citizenship"], columns=["decision"], aggfunc=count)

NameError: name 'count' is not defined

In [46]:
visa_decision = (pd.pivot_table(
    (visa.groupby("citizenship")["decision"]
    .value_counts()
    .to_frame()
    .rename(columns={"decision":"count"})), 
    index=["citizenship"], 
    columns=["decision"],
    values="count")
 .fillna(0)
)
visa_decision.head()

decision,N,Y
citizenship,,
AFGHANISTAN,1.0,3.0
ALBANIA,2.0,10.0
ALGERIA,2.0,1.0
ARGENTINA,22.0,67.0
ARMENIA,0.0,9.0


In [47]:
visa_decision["percent"] = visa_decision["Y"] / (visa_decision["N"] + visa_decision["Y"])
visa_decision.to_csv("")
visa_decision

decision,N,Y,percent
citizenship,,,
AFGHANISTAN,1.0,3.0,0.750000
ALBANIA,2.0,10.0,0.833333
ALGERIA,2.0,1.0,0.333333
ARGENTINA,22.0,67.0,0.752809
ARMENIA,0.0,9.0,1.000000
...,...,...,...
VIETNAM,9.0,55.0,0.859375
YEMEN,0.0,3.0,1.000000
YUGOSLAVIA,1.0,2.0,0.666667
